In [3]:
import pandas as pd
#we first read in a data set of one participant as an example
#it contains the raw impedance data coming from VU-AMS 5fs
#it also contains a column called Task_Label_2 which includes our task labels for the procedure

#data can also be read in another format as long as it is turned into a pandas dataframe format

#the dataframe should not be datetime indexed. if it is, drop the indexes before following with the next cells
df_acc = pd.read_hdf('df_acc.h5', 'df_acc')
df_acc


,MZR,Task_Label_2
0,0.333496,NaN
1,0.330078,NaN
2,0.330566,NaN
3,0.331055,NaN
4,0.333008,NaN
...,...,...
4139996,0.012695,NaN
4139997,0.007812,94.0
4139998,0.009277,NaN
4139999,0.009277,NaN


In [4]:
import numpy as np
import pandas as pd
import librosa
from scipy.signal import butter, filtfilt
from scipy.stats import kurtosis, skew, entropy
import math

def butterworth_highpass(data, cutoff_frequency=75, sampling_rate=1000, order=2):
    b, a = butter(order, cutoff_frequency, btype='high', fs=sampling_rate, analog=False)
    y = filtfilt(b, a, data)
    return y

tasks = ['12.0a', '12.0b', '12.0c', '12.0d', '12.0e', '12.0f', '14.0a', '14.0b', '14.0c', '14.0d', '14.0e',
         '14.0f', '16.0a', '16.0b', '16.0c', '16.0d', '16.0e', '16.0f', '18.0a', '18.0b', '18.0c', '20.0a',
         '20.0b', '20.0c', '23.0a', '23.0b', '23.0c', '25.0a', '25.0b', '25.0c', '28.0a', '28.0b', '28.0c',
         '30.0a', '30.0b', '30.0c', 34.0, 36.0, 38.0, 40.0, 42.0, 44.0, 47.0, 49.0, 51.0, 53.0, 55.0, 57.0, 60.0,
         62.0, 64.0, 66.0, 68.0, 70.0, '75.0a', '75.0b', '75.0c', '78.0a', '78.0b', '78.0c', 83.0, 85.0, 87.0,
         89.0, 91.0, 93.0]

# Initialize a list to store features for each task
all_features = []

for task in tasks:
    # Find the starting index of the task
    start_indices = df_acc[df_acc['Task_Label_2'] == task].index

    if len(start_indices) > 0:
        start_index = start_indices[0]

        # Select the 30000 samples starting from the first occurrence of the task
        acc_z = df_acc['MZR'][start_index:start_index + 30000]
        y_prefilter = acc_z.to_numpy()
        y = butterworth_highpass(y_prefilter)
        y_demeaned = y - np.mean(y)

        # Calculate the features for the entire 30000 samples
        sr = 1000
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=y_demeaned))
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=2048, hop_length=512))
        spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=2048, hop_length=512))
        spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=y, n_fft=2048, hop_length=512))
        spectral_rolloff_25 = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=2048, hop_length=512, roll_percent=0.25))
        spectral_rolloff_50 = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=2048, hop_length=512, roll_percent=0.50))
        spectral_rolloff_85 = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=2048, hop_length=512, roll_percent=0.85))
        rms = np.mean(librosa.feature.rms(y=y, frame_length=2048, hop_length=512))

        spectrum = np.fft.rfft(y)
        spectral_kurtosis = kurtosis(abs(spectrum))
        spectral_skewness = skew(abs(spectrum))
        spectral_entropy = entropy(abs(spectrum), base=2)
        spectral_variance = np.var(abs(spectrum))
        spectral_mean = np.sum(abs(spectrum)) / len(spectrum)

        def crest(spectrum):
            absSpectrum = abs(spectrum)
            spectralSum = np.sum(absSpectrum)
            maxFrequencyIndex = np.argmax(absSpectrum)
            maxSpectrum = absSpectrum[maxFrequencyIndex]
            return maxSpectrum / spectralSum

        crest_value = crest(spectrum)

        # Append features for the current task to the list
        features = [zcr, spectral_centroid, spectral_bandwidth, spectral_flatness,
                    spectral_rolloff_25, spectral_rolloff_50, spectral_rolloff_85, rms,
                    spectral_kurtosis, spectral_skewness, spectral_entropy,
                    spectral_variance, spectral_mean, crest_value, task]
        all_features.append(features)

# Create a DataFrame from the features
feature_names = ['AccZ_ZCR', 'AccZ_Spectral_Centroid', 'AccZ_Spectral_Bandwidth', 'AccZ_Spectral_Flatness',
                 'AccZ_Spectral_Rolloff_25', 'AccZ_Spectral_Rolloff_50', 'AccZ_Spectral_Rolloff_85', 'AccZ_RMS', 
                 'AccZ_Spectral_Kurtosis', 'AccZ_Spectral_Skewness', 'AccZ_Spectral_Entropy', 
                 'AccZ_Spectral_Variance', 'AccZ_Spectral_Mean', 'AccZ_Spectral_Crest', 'Task_Label']
df_features_acc_z = pd.DataFrame(all_features, columns=feature_names)
df_features_acc_z

,AccZ_ZCR,AccZ_Spectral_Centroid,AccZ_Spectral_Bandwidth,AccZ_Spectral_Flatness,AccZ_Spectral_Rolloff_25,AccZ_Spectral_Rolloff_50,AccZ_Spectral_Rolloff_85,AccZ_RMS,AccZ_Spectral_Kurtosis,AccZ_Spectral_Skewness,AccZ_Spectral_Entropy,AccZ_Spectral_Variance,AccZ_Spectral_Mean,AccZ_Spectral_Crest,Task_Label
0,0.522833,276.654754,127.054754,0.312058,168.812897,275.034759,430.970273,0.001250,0.178422,0.609950,13.565275,0.013598,0.185412,0.000266,12.0a
1,0.527890,277.632974,126.633506,0.308426,170.294293,276.689950,430.457164,0.001256,0.100004,0.592790,13.561486,0.013905,0.185970,0.000272,12.0b
2,0.526210,277.888339,126.214473,0.305534,171.709481,276.342360,430.622683,0.001245,0.147650,0.593708,13.557445,0.013684,0.184075,0.000269,12.0c
3,0.524431,278.614578,126.474076,0.301270,171.692929,276.524431,432.360633,0.001249,0.045634,0.591457,13.552324,0.014008,0.184124,0.000251,12.0d
4,0.525258,277.379462,125.846971,0.296397,170.964645,274.860964,430.738546,0.001252,0.133909,0.627825,13.543499,0.014328,0.183853,0.000263,12.0e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0.518455,271.400308,130.081253,0.318338,161.025225,268.703655,430.001986,0.001310,0.062607,0.539776,13.578510,0.014378,0.195773,0.000272,85.0
62,0.417994,233.622327,130.673815,0.224638,120.911679,211.574748,400.556144,0.002181,7.126885,2.129275,13.437786,0.061163,0.294910,0.000522,87.0
63,0.515029,268.261028,131.251416,0.324693,157.400358,265.343618,428.024033,0.001292,0.235906,0.550981,13.587132,0.013697,0.193916,0.000302,89.0
64,0.436201,243.704661,129.332988,0.245841,143.480204,220.098980,408.484507,0.001924,12.893274,2.567284,13.469895,0.046078,0.264425,0.000621,91.0


In [ ]:
#df_features_accgyro['Participant'] = 58682
import pandas as pd

# Create an ExcelWriter object
writer = pd.ExcelWriter('df_features_acc_z_58682.xlsx', engine='xlsxwriter')

# Convert the dataframe to an Excel sheet within the ExcelWriter object
df_features_acc_z.to_excel(writer, index=False, float_format='%.7f', sheet_name='Sheet1')

# Save the ExcelWriter object to disk
writer.save()